# 파이썬과 MySQL 연동

In [1]:
import pandas as pd
import pymysql

In [2]:
conn = pymysql.connect(host='127.0.0.1'
                , port=3306
                , user = 'root'
                , password='root1234'
                , db = 'mydb'
                , charset='utf8'
                ) 

In [3]:
cur = conn.cursor()

In [17]:
cur.execute('SHOW TABLES')

12

In [16]:
cur.fetchone() # 테이블 이름 하나씩 출력 (이터레이터)

('seoulbikedata',)

In [18]:
cur.fetchall() # 테이블 이름

(('author',),
 ('book',),
 ('cafe_order',),
 ('date_rainfall',),
 ('food_order',),
 ('food_product',),
 ('menu',),
 ('menu_sum_view',),
 ('order_info',),
 ('order_menu',),
 ('order_menu_date',),
 ('seoulbikedata',))

In [24]:
cur.execute('SELECT * FROM menu')

7

In [25]:
cur.fetchall()

((1, 'BIG 아메리카노', 1800),
 (2, '카페라떼', 2000),
 (3, '녹차라떼', 3000),
 (4, '카라멜마끼아또', 5000),
 (5, '바닐라라떼', 4500),
 (6, '카푸치노', 4000),
 (7, '에스프레소', 1500))

- 데이터 입력 / COMMIT

In [26]:
cur.execute('INSERT INTO menu VALUES (8, 미숫가루, 4000)') # 문자열을 그대로 넣으면 컬럼명으로 인식해서 에러남

OperationalError: (1054, "Unknown column '미숫가루' in 'field list'")

In [ ]:
cur.execute('INSERT INTO menu VALUES (8, "미숫가루", 4000)')

In [30]:
cur.execute('SELECT * FROM menu') # mysql db에 바로 적용되는 것이 아니라 commit을 해야 변경이됨
cur.fetchall()

((1, 'BIG 아메리카노', 1800),
 (2, '카페라떼', 2000),
 (3, '녹차라떼', 3000),
 (4, '카라멜마끼아또', 5000),
 (5, '바닐라라떼', 4500),
 (6, '카푸치노', 4000),
 (7, '에스프레소', 1500),
 (8, '미숫가루', 4000))

In [32]:
conn.commit() # 실제 DB에 반영

- ROLLBACK

In [33]:
cur.execute('INSERT INTO menu VALUES (9, "꿀라떼", 4500)')

1

In [34]:
cur.execute('SELECT * FROM menu')
cur.fetchall()

((1, 'BIG 아메리카노', 1800),
 (2, '카페라떼', 2000),
 (3, '녹차라떼', 3000),
 (4, '카라멜마끼아또', 5000),
 (5, '바닐라라떼', 4500),
 (6, '카푸치노', 4000),
 (7, '에스프레소', 1500),
 (8, '미숫가루', 4000),
 (9, '꿀라떼', 4500))

In [35]:
conn.rollback() # commit 전 명령 되돌리기

In [36]:
cur.execute('SELECT * FROM menu')
cur.fetchall()

((1, 'BIG 아메리카노', 1800),
 (2, '카페라떼', 2000),
 (3, '녹차라떼', 3000),
 (4, '카라멜마끼아또', 5000),
 (5, '바닐라라떼', 4500),
 (6, '카푸치노', 4000),
 (7, '에스프레소', 1500),
 (8, '미숫가루', 4000))

### 카페 주문에 적용

- 주문 확인

In [7]:
cur.execute('SELECT * FROM cafe_order')

4

In [8]:
cafe_order_list = cur.fetchall()
cafe_order_list

((1, datetime.date(2023, 5, 11)),
 (2, datetime.date(2023, 5, 12)),
 (3, datetime.date(2023, 5, 13)),
 (4, datetime.date(2023, 5, 14)))

In [39]:
cafe_order_list[0][1].year

2023

In [43]:
cafe_order_list[0][0]

1

In [9]:
# 파이썬 복습 !! cafe_order_list에서 [1, 2, 3, 4] 추출
order_id_list = [cafe_order[0] for cafe_order in cafe_order_list]
order_id_list

[1, 2, 3, 4]

In [47]:
import streamlit as st

In [10]:
order_id = 1

sql_script = f'''
SELECT om.order_id, m.name, m.price, om.menu_num, m.price * om.menu_num
  FROM menu AS m, order_menu AS om
 WHERE m.menu_id = om.menu_id AND om.order_id = {order_id}
'''


# cur.execute(sql_script)

In [49]:
# cur.fetchall()

((1, 'BIG 아메리카노', 1800, 2, 3600), (1, '카페라떼', 2000, 1, 2000))

In [12]:
cur.execute(sql_script)
col_names = ['주문번호', '메뉴', '가격', '수량', '가격x수량']
order_id_df = pd.DataFrame(cur.fetchall(), columns=col_names)
order_id_df

,주문번호,메뉴,가격,수량,가격x수량
0,1,BIG 아메리카노,1800,2,3600
1,1,카페라떼,2000,1,2000


In [13]:
# 총 금액 계산
total = order_id_df['가격x수량'].sum()
print(f'총 금액: {format(total, ",")} 원')

총 금액: 5,600 원


In [14]:
conn.close()